# Designing with Classes

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
os.getcwd()
os.chdir('/Users/fizz/Document/Notes/Python/codes')

In [ ]:
# Objects restored these ways retain both state and behavior
# pickle
import pickle
object = Someclass()
file = open(filename, 'wb')
pickle.dump(object, file)
file = open(filename, 'rb')
object = pickle.load(file)

# shelve
import shelve
object = Someclass()
dbase = shelve.open(filename)
dbase['key'] = object
dbase = shelve.open(filename)
object = dbase['key']

In [ ]:
# 'Wrapper' Proxy Objects
class Wrapper:
    def __init__(self, object):
        self.wrapped = object
    def __getattr__(self, attrname):
        print('Trace' + attrname)
        return getattr(self.object, attrname)

*  _X

* Name Mangling  
 Within a __class__ statement only, any names that starts with two underscores but don't end with two underscores are automatically expanded to include the name of the enclosing class at their front. For instance, a name like ___X__ within a class named __Spam__ is changed to __\_Spam\_\_X__ automatically.


In [4]:
class C1:
    def meth1(self): self.__X = 88
    def meth2(self): print(self.__X)
class C2:
    def metha(self): self.__X = 99
    def methb(self): print(self.__X)
class C3(C1, C2): pass
I = C3()
I.meth1(); I.metha()
print(I.__dict__)
I.meth2(); I.methb()

{'_C1__X': 88, '_C2__X': 99}
88
99


In [ ]:
class Super:
    def method(self): ...
class Tool:
    def __method(self): ...                   # Becomes _Tool__method
    def other(self): self.__method()
class Sub1(Tool, Super):
    def actions(self): self.method        # Runs Super.method as expected
class Sub2(Tool):
    def __init__(self): self.method = 99  # Doesn't break Tool.__method

* Sometimes, class-based designs require objects to be created in response to conditions that can't be predicted when a program is written. The factory design pattern allows such a deferred approach.

In [ ]:
def factory(aClass, *pargs, **kargs):
    return aClass(*pargs, **kargs)

* In __classic__ classes, the attribute search in all cases procecds depth-first all the way to the top of the inheritance tree, and then from left to right. This order is usually called DFLR, for its depth-first, left-to-right path.

* In __new-style__ classes, the attribute is usually as before, but in diamond patterns preoceeds across by tree levels before moving up, in a more breadth-first fashion. This order is usually called the new-syle MRO, for method resolution, though it's used for all attritbutes, not just methods.

In [26]:
# File testmixin.py
"""
Generic lister mixin tester: similar to transitive reloader in chapter 25, but passes a class
object to tester (not function), and testByNames adds loading of both module and class
by name strings here, in keeping with Chapter 31's factories pattern.
"""
import importlib
def tester(listerclass, sept=False):
    class Super:
        def __init__(self):
            self.data1 = 'spam'
        def ham(self):
            pass
    class Sub(Super, listerclass):
        def __init__(self):
            Super.__init__(self)
            self.data2 = 'eggs'
            self.data3 = 42
        def spam(self):
            pass
    instance = Sub()
    print(instance)
    if sept: print('-' * 80)
def testByNames(modname, classname, sept=False):
    modobject = importlib.import_module(modname)
    listerclass = getattr(modobject, classname)
    tester(listerclass, sept)

if __name__ == '__main__':
    testByNames('listinstance', 'ListInstance', True)
    testByNames('listinherited', 'ListInherited', True) 
    testByNames('listtree', 'ListTree', False)

"\nGeneric lister mixin tester: similar to transitive reloader in chapter 25, but passes a class\nobject to tester (not function), and testByNames adds loading of both module and class\nby name strings here, in keeping with Chapter 31's factories pattern.\n"

ModuleNotFoundError: No module named 'listinstance'

In [25]:
# Coding Mix-in Display Classes
class ListInstance:
    def __attrnames(self, indent=' ' * 4):
        # Notice how it escapes a %s with %% so that just one remains for the final
        # formatting operation at the end!!!
        result = 'Unders%s\n%s%%s\nOthers%s\n' % ('-' * 77, indent, '-' * 77)
        unders = []
        for attr in dir(self):
            if attr[:2] == '__' and attr[-2:] == '__':
                unders.append(attr)
            else:
                display = str(getattr(self, attr))[:82-len(indent)-len(attr)]
                result += '%s%s=%s\n' % (indent, attr, display)
        return result % ', '.join(unders)
    def __str__(self):
        return '<Instance of %s, address %s:\n%s>' % (
        self.__class__.__name__,
        id(self),
        self.__attrnames())
if __name__ == '__main__':
    import testmixin
    testmixin.tester(ListInstance)

<Instance of Sub, address 4463995592:
Unders-----------------------------------------------------------------------------
    __class__, __delattr__, __dict__, __dir__, __doc__, __eq__, __format__, __ge__, __getattribute__, __gt__, __hash__, __init__, __init_subclass__, __le__, __lt__, __module__, __ne__, __new__, __reduce__, __reduce_ex__, __repr__, __setattr__, __sizeof__, __str__, __subclasshook__, __weakref__
Others-----------------------------------------------------------------------------
    _ListInstance__attrnames=<bound method ListInstance.__attrnames of <testmixin.t
    data1=spam
    data2=eggs
    data3=42
    ham=<bound method tester.<locals>.Super.ham of <testmixin.tester.<locals>.Sub o
    spam=<bound method tester.<locals>.Sub.spam of <testmixin.tester.<locals>.Sub o
>


In [28]:
# FIle listtree.py
class ListTree:
    """
    Mix-in that returns an __str__ trace of the entire class tree and all its object's attrs at
    above self; run by print(), str() returns constructed string; uses __X attr names to avoid
    impacting clients; recurses to superclasses explicitly, uses str.format() for clarity;
    """
    def __attrnames(self, obj, indent):
        spaces = ' ' * (indent + 1)
        result = ''
        for attr in sorted(obj.__dict__):
            if attr.startswith('__') and attr.endswith('__'):
                result += spaces + '{0}\n'.format(attr)
            else:
                result += spaces + '{0}={1}\n'.format(attr, getattr(obj, attr))
        return result
    def __listclass(self, aClass, indent):
        dots = '.' * indent
        if aClass in self.__visited:
            return '\n{0}<Class {1}:, address {2}: (see above)>\n'.format(
                        dots,
                        aClass.__name__,
                        id(aClass))
        else:
            self.__visited[aClass] = True
            here = self.__attrnames(aClass, indent)
            above = ''
            for super in aClass.__bases__:
                above += self.__listclass(super, indent + 4)
            return '\n{0}<Class {1}, address {2}:\n{3}{4}{5}>\n'.format(
                dots,
                aClass.__name__,
                id(aClass),
                here, above,
                dots)
    def __str__(self):
            self.__visited = {}
            here = self.__attrnames(self, 0)
            above = self.__listclass(self.__class__, 4)
            return '<Instance of {0}, address {1}:\n{2}{3}>'.format(
                self.__class__.__name__,
                id(self),
                here, above)
if __name__ == '__main__':
    import testmixin
    testmixin.tester(ListTree)

<Instance of Sub, address 4463433376:
 _ListTree__visited={}
 data1=spam
 data2=eggs
 data3=42

....<Class Sub, address 140591623631032:
     __doc__
     __init__
     __module__
     spam=<function tester.<locals>.Sub.spam at 0x10a0a1f28>

........<Class Super, address 140591623630104:
         __dict__
         __doc__
         __init__
         __module__
         __weakref__
         ham=<function tester.<locals>.Super.ham at 0x10a0a1e18>

............<Class object, address 4424811816:
             __class__
             __delattr__
             __dir__
             __doc__
             __eq__
             __format__
             __ge__
             __getattribute__
             __gt__
             __hash__
             __init__
             __init_subclass__
             __le__
             __lt__
             __ne__
             __new__
             __reduce__
             __reduce_ex__
             __repr__
             __setattr__
             __sizeof__
             __str__
 